# Feature Selection and Engineering for PEMS Models

This notebook focuses on feature engineering and selection for machine learning models in the Predictive Energy Management System (PEMS).

## Key Analysis Areas:
1. **Feature Engineering**: Time-based, weather-derived, and energy-related features
2. **Feature Selection**: Statistical and ML-based feature importance
3. **Correlation Analysis**: Feature relationships and multicollinearity
4. **Target Variable Analysis**: PV production, heating demand, consumption patterns
5. **Model Input Preparation**: Cleaned and optimized feature sets for ML models


In [ ]:
import sys
import asyncio
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from scipy import stats
from sklearn.feature_selection import (
    SelectKBest, f_regression, mutual_info_regression,
    RFE, RFECV
)
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# Add parent directory to path
sys.path.append(str(Path().absolute().parent.parent))

from analysis.data_extraction import DataExtractor
from analysis.feature_engineering import FeatureEngineering
from config.settings import PEMSSettings

# Set up plotting
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
%matplotlib inline

# Suppress warnings for cleaner output
import warnings
warnings.filterwarnings('ignore')

## 1. Data Loading and Initial Feature Engineering

In [ ]:
# Initialize settings and data extractor
settings = PEMSSettings()
extractor = DataExtractor(settings)
feature_eng = FeatureEngineering()

# Define analysis period
end_date = datetime.now()
start_date = end_date - timedelta(days=365)  # 1 year

print(f"Analysis period: {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}")

In [ ]:
# Extract all relevant data
try:
    print("Extracting data...")
    weather_data = await extractor.extract_weather_data(start_date, end_date)
    pv_data = await extractor.extract_pv_data(start_date, end_date)
    room_data = await extractor.extract_room_data(start_date, end_date)
    consumption_data = await extractor.extract_consumption_data(start_date, end_date)
    relay_states = await extractor.extract_relay_states(start_date, end_date)
    price_data = await extractor.extract_energy_prices(start_date, end_date)
    
    print(f"Weather data: {weather_data.shape}")
    print(f"PV data: {pv_data.shape}")
    print(f"Room data: {len(room_data)} rooms")
    print(f"Consumption data: {consumption_data.shape}")
    print(f"Relay states: {len(relay_states)} relays")
    print(f"Price data: {price_data.shape if price_data is not None else 'None'}")
    
except Exception as e:
    print(f"Error loading data: {e}")
    # Load from saved files if extraction fails
    try:
        print("Loading from saved parquet files...")
        weather_data = extractor.load_from_parquet("weather_data")
        pv_data = extractor.load_from_parquet("pv_data")
        room_data = extractor.load_from_parquet("room_data")
        print("Data loaded successfully from saved files")
    except:
        print("Failed to load saved data. Please run data extraction first.")
        # Create dummy data for demonstration
        dates = pd.date_range(start_date, end_date, freq='15T')
        weather_data = pd.DataFrame({
            'temperature': np.random.normal(15, 10, len(dates)),
            'humidity': np.random.normal(70, 15, len(dates)),
            'solar_radiation': np.maximum(0, np.random.normal(200, 150, len(dates))),
            'wind_speed': np.random.exponential(3, len(dates))
        }, index=dates)
        
        pv_data = pd.DataFrame({
            'InputPower': np.maximum(0, weather_data['solar_radiation'] * 10 + np.random.normal(0, 100, len(dates)))
        }, index=dates)
        
        print("Using dummy data for demonstration")

## 2. Comprehensive Feature Engineering

In [ ]:
# Create comprehensive feature set
print("Creating feature set...")

# Start with aligned time index (hourly)
feature_df = pd.DataFrame(index=pd.date_range(start_date, end_date, freq='1H'))

# === TIME-BASED FEATURES ===
print("Adding time-based features...")
feature_df['hour'] = feature_df.index.hour
feature_df['day_of_week'] = feature_df.index.dayofweek
feature_df['month'] = feature_df.index.month
feature_df['quarter'] = feature_df.index.quarter
feature_df['day_of_year'] = feature_df.index.dayofyear
feature_df['week_of_year'] = feature_df.index.isocalendar().week

# Cyclical encoding for periodic features
feature_df['hour_sin'] = np.sin(2 * np.pi * feature_df['hour'] / 24)
feature_df['hour_cos'] = np.cos(2 * np.pi * feature_df['hour'] / 24)
feature_df['day_sin'] = np.sin(2 * np.pi * feature_df['day_of_week'] / 7)
feature_df['day_cos'] = np.cos(2 * np.pi * feature_df['day_of_week'] / 7)
feature_df['month_sin'] = np.sin(2 * np.pi * feature_df['month'] / 12)
feature_df['month_cos'] = np.cos(2 * np.pi * feature_df['month'] / 12)
feature_df['dayofyear_sin'] = np.sin(2 * np.pi * feature_df['day_of_year'] / 365)
feature_df['dayofyear_cos'] = np.cos(2 * np.pi * feature_df['day_of_year'] / 365)

# Weekend/weekday indicator
feature_df['is_weekend'] = (feature_df['day_of_week'] >= 5).astype(int)

# Season indicator
feature_df['season'] = feature_df['month'].map({
    12: 0, 1: 0, 2: 0,  # Winter
    3: 1, 4: 1, 5: 1,   # Spring
    6: 2, 7: 2, 8: 2,   # Summer
    9: 3, 10: 3, 11: 3  # Autumn
})

print(f"Time features added: {len([c for c in feature_df.columns if any(x in c for x in ['hour', 'day', 'month', 'season', 'weekend'])])}")

In [ ]:
# === WEATHER FEATURES ===
print("Adding weather features...")

if not weather_data.empty:
    # Resample weather data to hourly
    weather_hourly = weather_data.resample('1H').mean()
    
    # Basic weather features
    weather_cols = ['temperature', 'humidity', 'pressure', 'wind_speed', 
                   'cloud_cover', 'solar_radiation', 'precipitation']
    
    for col in weather_cols:
        if col in weather_hourly.columns:
            feature_df[col] = weather_hourly[col]
    
    # Derived weather features
    if 'temperature' in feature_df.columns:
        feature_df['temp_squared'] = feature_df['temperature'] ** 2
        feature_df['temp_cubed'] = feature_df['temperature'] ** 3
        
        # Heating/cooling degree days (base 18°C)
        feature_df['heating_degree_hours'] = np.maximum(0, 18 - feature_df['temperature'])
        feature_df['cooling_degree_hours'] = np.maximum(0, feature_df['temperature'] - 24)
        
        # Temperature ranges
        feature_df['temp_very_cold'] = (feature_df['temperature'] < 0).astype(int)
        feature_df['temp_cold'] = ((feature_df['temperature'] >= 0) & 
                                  (feature_df['temperature'] < 10)).astype(int)
        feature_df['temp_mild'] = ((feature_df['temperature'] >= 10) & 
                                  (feature_df['temperature'] < 20)).astype(int)
        feature_df['temp_warm'] = (feature_df['temperature'] >= 20).astype(int)
    
    # Wind chill factor
    if 'temperature' in feature_df.columns and 'wind_speed' in feature_df.columns:
        # Simplified wind chill calculation
        feature_df['wind_chill'] = (13.12 + 0.6215 * feature_df['temperature'] - 
                                   11.37 * (feature_df['wind_speed'] * 3.6) ** 0.16 + 
                                   0.3965 * feature_df['temperature'] * 
                                   (feature_df['wind_speed'] * 3.6) ** 0.16)
    
    # Solar position approximation
    if 'solar_radiation' in feature_df.columns:
        feature_df['solar_elevation'] = np.maximum(0, 
            np.sin(2 * np.pi * (feature_df['day_of_year'] - 81) / 365) * 
            np.sin(2 * np.pi * (feature_df['hour'] - 12) / 24) * 0.4)
        
        # Clear sky index (actual vs theoretical maximum)
        theoretical_max = feature_df['solar_elevation'] * 1000  # W/m²
        feature_df['clear_sky_index'] = np.where(theoretical_max > 0, 
                                                feature_df['solar_radiation'] / theoretical_max, 0)
        feature_df['clear_sky_index'] = np.clip(feature_df['clear_sky_index'], 0, 1.2)
    
    print(f"Weather features added: {len([c for c in feature_df.columns if any(x in c for x in ['temp', 'humid', 'wind', 'solar', 'cloud', 'precip', 'pressure'])])}")
else:
    print("No weather data available")

In [ ]:
# === ENERGY FEATURES ===
print("Adding energy features...")

# PV features
if not pv_data.empty:
    pv_hourly = pv_data.resample('1H').agg({
        'InputPower': ['mean', 'max', 'std'],
        'OutputPower': ['mean', 'max'] if 'OutputPower' in pv_data.columns else None
    }).fillna(0)
    
    if ('InputPower', 'mean') in pv_hourly.columns:
        feature_df['pv_power_mean'] = pv_hourly[('InputPower', 'mean')]
        feature_df['pv_power_max'] = pv_hourly[('InputPower', 'max')]
        feature_df['pv_power_std'] = pv_hourly[('InputPower', 'std')].fillna(0)
        
        # PV capacity factor
        feature_df['pv_capacity_factor'] = feature_df['pv_power_mean'] / 10000  # Assuming 10kW system
        
        # PV production indicators
        feature_df['pv_producing'] = (feature_df['pv_power_mean'] > 50).astype(int)
        feature_df['pv_peak_production'] = (feature_df['pv_power_mean'] > 7000).astype(int)
    
    print(f"PV features added: {len([c for c in feature_df.columns if 'pv' in c])}")

# Consumption features
if not consumption_data.empty:
    consumption_hourly = consumption_data.resample('1H').agg(['mean', 'max', 'std']).fillna(0)
    
    if len(consumption_hourly.columns) > 0:
        feature_df['consumption_mean'] = consumption_hourly.iloc[:, 0]  # First column mean
        if len(consumption_hourly.columns) > 1:
            feature_df['consumption_max'] = consumption_hourly.iloc[:, 1]
        if len(consumption_hourly.columns) > 2:
            feature_df['consumption_std'] = consumption_hourly.iloc[:, 2]
    
    print(f"Consumption features added: {len([c for c in feature_df.columns if 'consumption' in c])}")

# Relay/heating features
if relay_states:
    total_relays = pd.DataFrame()
    for room, relay_data in relay_states.items():
        if not relay_data.empty:
            if total_relays.empty:
                total_relays = relay_data.resample('1H').mean()
                total_relays.columns = ['total_heating_demand']
            else:
                total_relays['total_heating_demand'] += relay_data.resample('1H').mean().iloc[:, 0]
    
    if not total_relays.empty:
        feature_df['heating_demand'] = total_relays['total_heating_demand']
        feature_df['heating_on'] = (feature_df['heating_demand'] > 0.1).astype(int)
        feature_df['heating_high'] = (feature_df['heating_demand'] > 8).astype(int)  # More than half relays
    
    print(f"Heating features added: {len([c for c in feature_df.columns if 'heating' in c])}")

# Price features
if price_data is not None and not price_data.empty:
    price_hourly = price_data.resample('1H').mean()
    
    if 'price' in price_hourly.columns:
        feature_df['energy_price'] = price_hourly['price']
        
        # Price-based features
        price_median = feature_df['energy_price'].median()
        feature_df['price_high'] = (feature_df['energy_price'] > price_median * 1.5).astype(int)
        feature_df['price_low'] = (feature_df['energy_price'] < price_median * 0.5).astype(int)
        
        # Price relative to daily average
        daily_avg_price = feature_df['energy_price'].resample('D').mean()
        feature_df['price_vs_daily_avg'] = (feature_df['energy_price'] / 
                                           feature_df.index.to_series().dt.date.map(daily_avg_price))
    
    print(f"Price features added: {len([c for c in feature_df.columns if 'price' in c])}")

print(f"\nTotal features created: {len(feature_df.columns)}")
print(f"Data points: {len(feature_df)}")
print(f"Memory usage: {feature_df.memory_usage(deep=True).sum() / 1024**2:.1f} MB")

## 3. Lag and Rolling Window Features

In [ ]:
# === LAG FEATURES ===
print("Adding lag and rolling window features...")

# Define key variables for lag features
lag_variables = ['temperature', 'solar_radiation', 'pv_power_mean', 'heating_demand']
lag_periods = [1, 2, 3, 6, 12, 24]  # 1h, 2h, 3h, 6h, 12h, 24h

for var in lag_variables:
    if var in feature_df.columns:
        # Lag features
        for lag in lag_periods:
            feature_df[f'{var}_lag_{lag}h'] = feature_df[var].shift(lag)
        
        # Rolling window features
        for window in [3, 6, 12, 24]:
            feature_df[f'{var}_roll_mean_{window}h'] = feature_df[var].rolling(window).mean()
            feature_df[f'{var}_roll_std_{window}h'] = feature_df[var].rolling(window).std()
            feature_df[f'{var}_roll_max_{window}h'] = feature_df[var].rolling(window).max()
            feature_df[f'{var}_roll_min_{window}h'] = feature_df[var].rolling(window).min()

# Weekly and daily patterns
for var in ['temperature', 'pv_power_mean', 'heating_demand']:
    if var in feature_df.columns:
        # Same hour yesterday
        feature_df[f'{var}_same_hour_yesterday'] = feature_df[var].shift(24)
        
        # Same hour last week
        feature_df[f'{var}_same_hour_last_week'] = feature_df[var].shift(24 * 7)
        
        # Average for this hour of week (historical)
        hour_week_avg = feature_df.groupby([feature_df.index.hour, feature_df.index.dayofweek])[var].expanding().mean()
        feature_df[f'{var}_hour_week_avg'] = hour_week_avg.droplevel([0, 1])

print(f"Lag and rolling features added. Total features: {len(feature_df.columns)}")

## 4. Feature Quality Assessment

In [ ]:
# === FEATURE QUALITY ASSESSMENT ===
print("Assessing feature quality...")

# Remove features with too many missing values
missing_threshold = 0.3  # 30% missing values
missing_ratios = feature_df.isnull().sum() / len(feature_df)
features_to_drop = missing_ratios[missing_ratios > missing_threshold].index.tolist()

print(f"Features with >30% missing values (to be dropped): {len(features_to_drop)}")
if features_to_drop:
    print(f"Dropping: {features_to_drop[:5]}{'...' if len(features_to_drop) > 5 else ''}")
    feature_df = feature_df.drop(columns=features_to_drop)

# Remove features with zero variance
numeric_features = feature_df.select_dtypes(include=[np.number]).columns
zero_variance_features = []
for col in numeric_features:
    if feature_df[col].std() == 0:
        zero_variance_features.append(col)

print(f"Zero variance features (to be dropped): {len(zero_variance_features)}")
if zero_variance_features:
    feature_df = feature_df.drop(columns=zero_variance_features)

# Feature correlation analysis
correlation_matrix = feature_df[numeric_features].corr()

# Find highly correlated feature pairs
high_corr_pairs = []
correlation_threshold = 0.95

for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        if abs(correlation_matrix.iloc[i, j]) > correlation_threshold:
            high_corr_pairs.append((
                correlation_matrix.columns[i],
                correlation_matrix.columns[j],
                correlation_matrix.iloc[i, j]
            ))

print(f"Highly correlated feature pairs (>0.95): {len(high_corr_pairs)}")

# Remove one feature from each highly correlated pair
features_to_remove = set()
for feat1, feat2, corr in high_corr_pairs:
    # Keep the feature with fewer missing values
    if feature_df[feat1].isnull().sum() > feature_df[feat2].isnull().sum():
        features_to_remove.add(feat1)
    else:
        features_to_remove.add(feat2)

if features_to_remove:
    print(f"Removing {len(features_to_remove)} highly correlated features")
    feature_df = feature_df.drop(columns=list(features_to_remove))

# Final feature summary
print(f"\nFinal feature set: {len(feature_df.columns)} features")
print(f"Missing value summary:")
remaining_missing = feature_df.isnull().sum()
if remaining_missing.sum() > 0:
    print(remaining_missing[remaining_missing > 0].sort_values(ascending=False).head(10))
else:
    print("No missing values in final feature set")

## 5. Target Variable Definition and Preparation

In [ ]:
# === TARGET VARIABLE PREPARATION ===
print("Preparing target variables for different prediction tasks...")

# Clean the feature dataframe
clean_features = feature_df.dropna()

# Define prediction targets
targets = {}

# 1. PV Production Prediction (next hour)
if 'pv_power_mean' in clean_features.columns:
    targets['pv_next_hour'] = clean_features['pv_power_mean'].shift(-1)
    print(f"PV prediction target: {targets['pv_next_hour'].notna().sum()} valid samples")

# 2. Heating Demand Prediction (next hour)
if 'heating_demand' in clean_features.columns:
    targets['heating_next_hour'] = clean_features['heating_demand'].shift(-1)
    print(f"Heating prediction target: {targets['heating_next_hour'].notna().sum()} valid samples")

# 3. Total Consumption Prediction
if 'consumption_mean' in clean_features.columns:
    targets['consumption_next_hour'] = clean_features['consumption_mean'].shift(-1)
    print(f"Consumption prediction target: {targets['consumption_next_hour'].notna().sum()} valid samples")

# 4. Net Energy Balance (PV - Consumption)
if 'pv_power_mean' in clean_features.columns and 'consumption_mean' in clean_features.columns:
    net_balance = clean_features['pv_power_mean'] - clean_features['consumption_mean']
    targets['net_balance_next_hour'] = net_balance.shift(-1)
    print(f"Net balance prediction target: {targets['net_balance_next_hour'].notna().sum()} valid samples")

# Display target variable distributions
if targets:
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    axes = axes.flatten()
    
    for i, (target_name, target_data) in enumerate(targets.items()):
        if i < len(axes) and target_data.notna().sum() > 0:
            valid_data = target_data.dropna()
            
            axes[i].hist(valid_data, bins=50, alpha=0.7, edgecolor='black')
            axes[i].set_title(f'{target_name.replace("_", " ").title()} Distribution')
            axes[i].set_ylabel('Frequency')
            axes[i].grid(True, alpha=0.3)
            
            # Add statistics
            stats_text = f"Mean: {valid_data.mean():.1f}\nStd: {valid_data.std():.1f}\nSamples: {len(valid_data)}"
            axes[i].text(0.7, 0.7, stats_text, transform=axes[i].transAxes, 
                        bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    # Hide unused subplots
    for i in range(len(targets), len(axes)):
        axes[i].set_visible(False)
    
    plt.tight_layout()
    plt.show()
else:
    print("No valid target variables could be created")

## 6. Feature Selection Methods

In [ ]:
# === FEATURE SELECTION ===
print("Performing feature selection analysis...")

# Choose primary target for feature selection
primary_target_name = 'pv_next_hour' if 'pv_next_hour' in targets else list(targets.keys())[0] if targets else None

if primary_target_name and targets[primary_target_name].notna().sum() > 100:
    print(f"Using '{primary_target_name}' as primary target for feature selection")
    
    # Prepare data for feature selection
    target = targets[primary_target_name]
    
    # Get features excluding target-related columns
    feature_columns = [col for col in clean_features.columns 
                      if not any(exclude in col.lower() for exclude in ['target', 'next_hour', 'future'])]
    
    X = clean_features[feature_columns]
    y = target
    
    # Create common valid index
    valid_idx = X.index.intersection(y.dropna().index)
    X_valid = X.loc[valid_idx].dropna()
    y_valid = y.loc[X_valid.index]
    
    print(f"Valid samples for feature selection: {len(X_valid)}")
    print(f"Number of features to evaluate: {len(X_valid.columns)}")
    
    if len(X_valid) > 100 and len(X_valid.columns) > 5:
        # Split data for evaluation
        X_train, X_test, y_train, y_test = train_test_split(
            X_valid, y_valid, test_size=0.2, random_state=42
        )
        
        # Scale features
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)
        
        # === 1. Univariate Feature Selection ===
        print("\n1. Univariate Feature Selection (F-statistic)...")
        
        selector_f = SelectKBest(score_func=f_regression, k=min(20, len(X_train.columns)))
        X_train_f = selector_f.fit_transform(X_train_scaled, y_train)
        
        # Get feature scores
        f_scores = pd.DataFrame({
            'feature': X_train.columns,
            'f_score': selector_f.scores_,
            'p_value': selector_f.pvalues_
        }).sort_values('f_score', ascending=False)
        
        print(f"Top 10 features by F-statistic:")
        display(f_scores.head(10))
        
        # === 2. Mutual Information ===
        print("\n2. Mutual Information Feature Selection...")
        
        # Use subset of data for faster computation
        subset_size = min(5000, len(X_train))
        subset_idx = np.random.choice(len(X_train), subset_size, replace=False)
        
        mi_scores = mutual_info_regression(
            X_train_scaled[subset_idx], y_train.iloc[subset_idx], 
            random_state=42
        )
        
        mi_df = pd.DataFrame({
            'feature': X_train.columns,
            'mutual_info': mi_scores
        }).sort_values('mutual_info', ascending=False)
        
        print(f"Top 10 features by Mutual Information:")
        display(mi_df.head(10))
        
        # === 3. Random Forest Feature Importance ===
        print("\n3. Random Forest Feature Importance...")
        
        rf = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
        rf.fit(X_train_scaled, y_train)
        
        rf_importance = pd.DataFrame({
            'feature': X_train.columns,
            'importance': rf.feature_importances_
        }).sort_values('importance', ascending=False)
        
        print(f"Top 10 features by Random Forest Importance:")
        display(rf_importance.head(10))
        
        # Model performance with all features
        y_pred_rf = rf.predict(X_test_scaled)
        rf_r2 = r2_score(y_test, y_pred_rf)
        rf_rmse = np.sqrt(mean_squared_error(y_test, y_pred_rf))
        
        print(f"\nRandom Forest Performance (all features):")
        print(f"R² Score: {rf_r2:.4f}")
        print(f"RMSE: {rf_rmse:.2f}")
        
    else:
        print("Insufficient data for comprehensive feature selection")
        f_scores = mi_df = rf_importance = None
        
else:
    print("No suitable target variable available for feature selection")
    f_scores = mi_df = rf_importance = None

## 7. Feature Importance Visualization

In [ ]:
# === FEATURE IMPORTANCE VISUALIZATION ===
if f_scores is not None and mi_df is not None and rf_importance is not None:
    print("Creating feature importance visualizations...")
    
    fig, axes = plt.subplots(2, 2, figsize=(18, 12))
    
    # 1. F-statistic scores
    top_f = f_scores.head(15)
    axes[0, 0].barh(range(len(top_f)), top_f['f_score'], alpha=0.7)
    axes[0, 0].set_yticks(range(len(top_f)))
    axes[0, 0].set_yticklabels(top_f['feature'], fontsize=8)
    axes[0, 0].set_xlabel('F-Statistic Score')
    axes[0, 0].set_title('Top 15 Features by F-Statistic')
    axes[0, 0].invert_yaxis()
    axes[0, 0].grid(True, alpha=0.3)
    
    # 2. Mutual Information scores
    top_mi = mi_df.head(15)
    axes[0, 1].barh(range(len(top_mi)), top_mi['mutual_info'], alpha=0.7, color='orange')
    axes[0, 1].set_yticks(range(len(top_mi)))
    axes[0, 1].set_yticklabels(top_mi['feature'], fontsize=8)
    axes[0, 1].set_xlabel('Mutual Information Score')
    axes[0, 1].set_title('Top 15 Features by Mutual Information')
    axes[0, 1].invert_yaxis()
    axes[0, 1].grid(True, alpha=0.3)
    
    # 3. Random Forest importance
    top_rf = rf_importance.head(15)
    axes[1, 0].barh(range(len(top_rf)), top_rf['importance'], alpha=0.7, color='green')
    axes[1, 0].set_yticks(range(len(top_rf)))
    axes[1, 0].set_yticklabels(top_rf['feature'], fontsize=8)
    axes[1, 0].set_xlabel('Random Forest Importance')
    axes[1, 0].set_title('Top 15 Features by Random Forest')
    axes[1, 0].invert_yaxis()
    axes[1, 0].grid(True, alpha=0.3)
    
    # 4. Combined ranking
    # Normalize scores and combine
    combined_ranking = pd.DataFrame({'feature': X_train.columns})
    
    # Add normalized ranks (lower rank = better)
    f_ranks = f_scores.reset_index()[['feature']].merge(
        f_scores.reset_index()[['feature']].reset_index().rename(columns={'index': 'f_rank'}),
        on='feature'
    )
    
    mi_ranks = mi_df.reset_index()[['feature']].merge(
        mi_df.reset_index()[['feature']].reset_index().rename(columns={'index': 'mi_rank'}),
        on='feature'
    )
    
    rf_ranks = rf_importance.reset_index()[['feature']].merge(
        rf_importance.reset_index()[['feature']].reset_index().rename(columns={'index': 'rf_rank'}),
        on='feature'
    )
    
    # Merge all rankings
    combined = combined_ranking.merge(f_ranks, on='feature', how='left')
    combined = combined.merge(mi_ranks, on='feature', how='left')
    combined = combined.merge(rf_ranks, on='feature', how='left')
    
    # Fill missing ranks with max rank + 1
    max_rank = len(combined)
    combined = combined.fillna(max_rank)
    
    # Calculate average rank
    combined['avg_rank'] = (combined['f_rank'] + combined['mi_rank'] + combined['rf_rank']) / 3
    combined = combined.sort_values('avg_rank')
    
    top_combined = combined.head(15)
    axes[1, 1].barh(range(len(top_combined)), 1 / (top_combined['avg_rank'] + 1), alpha=0.7, color='red')
    axes[1, 1].set_yticks(range(len(top_combined)))
    axes[1, 1].set_yticklabels(top_combined['feature'], fontsize=8)
    axes[1, 1].set_xlabel('Combined Importance (1/avg_rank)')
    axes[1, 1].set_title('Top 15 Features by Combined Ranking')
    axes[1, 1].invert_yaxis()
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Print top features summary
    print("\n=== TOP 10 FEATURES BY COMBINED RANKING ===")
    top_10_combined = combined.head(10)
    for i, row in top_10_combined.iterrows():
        print(f"{int(row.name)+1:2d}. {row['feature']:<35} (avg rank: {row['avg_rank']:.1f})")
    
else:
    print("Feature importance visualization skipped - insufficient data")

## 8. Optimal Feature Set Selection

In [ ]:
# === OPTIMAL FEATURE SET SELECTION ===
if 'combined' in locals() and len(X_valid) > 100:
    print("Finding optimal feature set size...")
    
    # Test different feature set sizes
    feature_counts = [5, 10, 15, 20, 30, 50, min(100, len(combined))]
    performance_results = []
    
    for n_features in feature_counts:
        if n_features <= len(combined):
            # Select top N features
            top_features = combined.head(n_features)['feature'].tolist()
            
            # Train model with selected features
            X_train_selected = X_train[top_features]
            X_test_selected = X_test[top_features]
            
            # Scale selected features
            scaler_selected = StandardScaler()
            X_train_scaled_selected = scaler_selected.fit_transform(X_train_selected)
            X_test_scaled_selected = scaler_selected.transform(X_test_selected)
            
            # Train Random Forest
            rf_selected = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
            rf_selected.fit(X_train_scaled_selected, y_train)
            
            # Evaluate
            y_pred_selected = rf_selected.predict(X_test_scaled_selected)
            r2_selected = r2_score(y_test, y_pred_selected)
            rmse_selected = np.sqrt(mean_squared_error(y_test, y_pred_selected))
            
            performance_results.append({
                'n_features': n_features,
                'r2_score': r2_selected,
                'rmse': rmse_selected
            })
            
            print(f"Features: {n_features:3d}, R²: {r2_selected:.4f}, RMSE: {rmse_selected:.2f}")
    
    # Plot performance vs number of features
    if performance_results:
        perf_df = pd.DataFrame(performance_results)
        
        fig, axes = plt.subplots(1, 2, figsize=(12, 5))
        
        # R² Score
        axes[0].plot(perf_df['n_features'], perf_df['r2_score'], 'o-', linewidth=2, markersize=8)
        axes[0].set_xlabel('Number of Features')
        axes[0].set_ylabel('R² Score')
        axes[0].set_title('Model Performance vs Feature Count')
        axes[0].grid(True, alpha=0.3)
        
        # Find optimal point (elbow in performance)
        best_idx = perf_df['r2_score'].idxmax()
        axes[0].axvline(perf_df.loc[best_idx, 'n_features'], color='red', linestyle='--', 
                       label=f'Best: {perf_df.loc[best_idx, "n_features"]} features')
        axes[0].legend()
        
        # RMSE
        axes[1].plot(perf_df['n_features'], perf_df['rmse'], 'o-', linewidth=2, markersize=8, color='orange')
        axes[1].set_xlabel('Number of Features')
        axes[1].set_ylabel('RMSE')
        axes[1].set_title('Model Error vs Feature Count')
        axes[1].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
        
        # Recommend optimal feature set
        optimal_n_features = perf_df.loc[best_idx, 'n_features']
        optimal_r2 = perf_df.loc[best_idx, 'r2_score']
        
        print(f"\n=== OPTIMAL FEATURE SET RECOMMENDATION ===")
        print(f"Recommended number of features: {optimal_n_features}")
        print(f"Expected R² performance: {optimal_r2:.4f}")
        
        # Show optimal feature list
        optimal_features = combined.head(optimal_n_features)['feature'].tolist()
        print(f"\nOptimal feature list:")
        for i, feature in enumerate(optimal_features, 1):
            print(f"{i:2d}. {feature}")
        
        # Save optimal feature set
        optimal_feature_set = {
            'target_variable': primary_target_name,
            'n_features': optimal_n_features,
            'expected_r2': optimal_r2,
            'features': optimal_features,
            'selection_method': 'combined_ranking',
            'created_date': datetime.now().isoformat()
        }
        
        print(f"\nOptimal feature set ready for model training!")
        
else:
    print("Optimal feature set selection skipped - insufficient data")

## 9. Feature Categories and Engineering Summary

In [ ]:
# === FEATURE CATEGORIZATION AND SUMMARY ===
print("=== FEATURE ENGINEERING SUMMARY ===")

# Categorize features
feature_categories = {
    'Time-based': [col for col in feature_df.columns if any(x in col for x in ['hour', 'day', 'month', 'season', 'weekend', 'quarter'])],
    'Weather': [col for col in feature_df.columns if any(x in col for x in ['temp', 'humid', 'wind', 'solar', 'cloud', 'precip', 'pressure'])],
    'Energy': [col for col in feature_df.columns if any(x in col for x in ['pv', 'consumption', 'heating', 'price', 'balance'])],
    'Lag/Rolling': [col for col in feature_df.columns if any(x in col for x in ['lag', 'roll', 'yesterday', 'last_week', 'avg'])],
    'Derived': [col for col in feature_df.columns if any(x in col for x in ['squared', 'cubed', 'degree', 'chill', 'index', 'factor'])]
}

print("\nFeature categories:")
for category, features in feature_categories.items():
    print(f"{category:<15}: {len(features):3d} features")
    if len(features) > 0:
        example_features = features[:3]
        print(f"                 Examples: {', '.join(example_features)}{'...' if len(features) > 3 else ''}")

# Feature quality metrics
print(f"\nFeature quality metrics:")
print(f"Total features created: {len(feature_df.columns)}")
print(f"Features after cleaning: {len(clean_features.columns)}")
print(f"Data completeness: {(1 - clean_features.isnull().sum().sum() / (len(clean_features) * len(clean_features.columns))):.1%}")
print(f"Time span: {clean_features.index.min()} to {clean_features.index.max()}")
print(f"Data points: {len(clean_features):,}")

# Engineering recommendations
print(f"\n=== FEATURE ENGINEERING RECOMMENDATIONS ===")
recommendations = []

if 'combined' in locals():
    top_feature_types = []
    for feature in combined.head(10)['feature']:
        for category, features in feature_categories.items():
            if feature in features:
                top_feature_types.append(category)
                break
    
    from collections import Counter
    type_counts = Counter(top_feature_types)
    most_important_type = type_counts.most_common(1)[0][0] if type_counts else "Unknown"
    
    recommendations.append(f"Focus on {most_important_type.lower()} features - they dominate top rankings")

if len([col for col in feature_df.columns if 'lag' in col]) > 0:
    recommendations.append("Lag features are valuable - consider expanding lag periods for seasonal patterns")

if len([col for col in feature_df.columns if any(x in col for x in ['sin', 'cos'])]) > 0:
    recommendations.append("Cyclical encoding is effective - apply to more periodic variables")

if 'weather' in str(feature_categories).lower():
    recommendations.append("Weather derivatives show value - consider more complex weather interactions")

recommendations.append("Regularly retrain feature selection with new data to adapt to changing patterns")
recommendations.append("Consider domain-specific features (heating curves, PV degradation, seasonal adjustments)")

for i, rec in enumerate(recommendations, 1):
    print(f"{i}. {rec}")

print(f"\nFeature engineering analysis completed successfully!")
print(f"Ready for model training with optimized feature set.")
print(f"Report generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")